# RFP RAG PoC — AI Coding Agent

This notebook contains a self-contained Proof-of-Concept (PoC) for a human-centric Retrieval-Augmented Generation (RAG) system for RFP handling. The PoC runs in Google Colab, uses **OpenAI 3.5 Turbo** for model calls, persists Knowledge Modules to a **JSON file** (`module_store.json`), accepts up to **two** uploaded past successful RFPs for ingestion, and exposes a minimal FastAPI backend + simple HTML UI.

## 1. Project Setup and Dependencies

In [ ]:
%pip install fastapi uvicorn python-multipart openai pdfplumber nest-asyncio

In [ ]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')

## 2. Core Logic and Agent Implementation

In [ ]:
import json
import hashlib
import datetime
from typing import List, Dict, Any
import re
import openai

# -- Knowledge Module Schema --
MODULE_SCHEMA = {
    "type": "object",
    "properties": {
        "module_id": {"type": "string"},
        "title": {"type": "string"},
        "content": {"type": "string"},
        "source_document_id": {"type": "string"},
        "source_excerpt": {"type": "string"},
        "confidence_level": {"type": "number", "minimum": 0.0, "maximum": 1.0},
        "created_at": {"type": "string", "format": "date-time"},
        "tags": {"type": "array", "items": {"type": "string"}}
    },
    "required": ["module_id", "title", "content", "source_document_id", "created_at"]
}

LOGS_FILE = 'logs.txt'

def log_agent_call(agent_name: str, inputs: Dict[str, Any], outputs: Dict[str, Any], model_call: Dict = None):
    log_entry = {
        'agent': agent_name,
        'input_summary': {k: (v[:100] + '...' if isinstance(v, str) and len(v) > 100 else v) for k, v in inputs.items()},
        'output_summary': {k: (v[:100] + '...' if isinstance(v, str) and len(v) > 100 else v) for k, v in outputs.items()},
        'timestamp': datetime.datetime.utcnow().isoformat(),
    }
    if model_call:
        log_entry['model_call'] = model_call
    with open(LOGS_FILE, 'a') as f:
        f.write(json.dumps(log_entry) + '\n')

def generate_module_id(content: str) -> str:
    """Generates a deterministic module_id."""
    sha_hash = hashlib.sha1(content.encode()).hexdigest()[:8]
    timestamp = datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S')
    return f"KM-{sha_hash}-{timestamp}"

def distill_document_agent(raw_document_text: str, source_document_id: str) -> List[dict]:
    """Extracts Knowledge Modules from a raw document."""
    # Simple paragraph splitting
    paragraphs = [p.strip() for p in raw_document_text.split('\n\n') if len(p.strip()) > 100]
    modules = []
    for p in paragraphs:
        module_id = generate_module_id(p)
        module = {
            "module_id": module_id,
            "title": p.split('\n')[0][:50], # Simple title generation
            "content": p,
            "source_document_id": source_document_id,
            "source_excerpt": p[:300],
            "confidence_level": 0.8, # Default confidence
            "created_at": datetime.datetime.utcnow().isoformat(),
            "tags": [] # Tags can be added later
        }
        modules.append(module)
    
    log_agent_call('distill_document_agent', {'raw_document_text': raw_document_text, 'source_document_id': source_document_id}, {'modules_created': len(modules)})
    return modules

def retrieve_modules_agent(question: str, module_store: List[dict]) -> dict:
    """Retrieves relevant modules based on a question."""
    selected_modules = []
    rationales = {}
    question_keywords = set(re.findall(r'\w+', question.lower()))

    for module in module_store:
        content_keywords = set(re.findall(r'\w+', module['content'].lower()))
        if question_keywords.intersection(content_keywords):
            selected_modules.append(module['module_id'])
            rationales[module['module_id']] = 'Keyword overlap found.'
    
    status = "found" if selected_modules else "none"
    message = "" if status == "found" else "No relevant modules found"
    
    result = {
      "selected_module_ids": selected_modules,
      "rationales": rationales,
      "status": status,
      "message": message
    }
    log_agent_call('retrieve_modules_agent', {'question': question, 'module_count': len(module_store)}, result)
    return result

def assemble_answer_agent(question: str, selected_modules: List[dict]) -> dict:
    """Assembles an answer from selected modules."""
    if not selected_modules:
        return {"status": "insufficient", "reason": "No modules provided."}

    context = "\n\n".join([f"Source (module_id: {m['module_id']}):\n{m['content']}" for m in selected_modules])
    prompt = f"""
    You are an AI assistant for RFP response generation. Your task is to answer the user's question using ONLY the provided text sources. Do not add any information that is not present in the sources.
    Each sentence in your answer must be traceable to one or more of the provided module_ids.
    Respond with a JSON object containing two keys: 'draft_answer' and 'sentence_traces'.
    'draft_answer' should be a string containing the answer.
    'sentence_traces' should be a list of objects, where each object has 'sentence' and 'module_ids' keys.

    User Question: {question}

    Sources:
    {context}
    """
    try:
        client = openai.OpenAI()
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You must generate a JSON object with 'draft_answer' and 'sentence_traces' keys."},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}
        )
        model_output = json.loads(response.choices[0].message.content)
        
        # Basic validation of the model's output
        if 'draft_answer' not in model_output or 'sentence_traces' not in model_output:
             raise ValueError("Model output missing required keys")

        used_module_ids = list(set(mid for trace in model_output.get('sentence_traces', []) for mid in trace.get('module_ids', [])))

        result = {
          "draft_answer": model_output['draft_answer'],
          "sentence_traces": model_output['sentence_traces'],
          "used_module_ids": used_module_ids,
          "status": "ok",
          "reason": ""
        }
        log_agent_call('assemble_answer_agent', {'question': question, 'module_count': len(selected_modules)}, result, {'prompt': prompt, 'response': model_output})
        return result

    except Exception as e:
        result = {"status": "insufficient", "reason": f"Failed to generate answer: {e}"}
        log_agent_call('assemble_answer_agent', {'question': question, 'module_count': len(selected_modules)}, result)
        return result


## 3. FastAPI Application and UI

In [ ]:
from fastapi import FastAPI, File, UploadFile, HTTPException, Form
from fastapi.responses import HTMLResponse, JSONResponse
from pydantic import BaseModel
import pdfplumber
import io

app = FastAPI()

MODULE_STORE_FILE = 'module_store.json'

class QueryRequest(BaseModel):
    question: str

def read_module_store() -> List[dict]:
    if not os.path.exists(MODULE_STORE_FILE):
        return []
    with open(MODULE_STORE_FILE, 'r') as f:
        return json.load(f)

def write_module_store(data: List[dict]):
    with open(MODULE_STORE_FILE, 'w') as f:
        json.dump(data, f, indent=2)

html_ui = """
<!DOCTYPE html>
<html>
<head>
    <title>RFP RAG PoC</title>
    <style>
        body { font-family: sans-serif; margin: 2em; }
        .container { max-width: 800px; margin: auto; }
        .panel { border: 1px solid #ccc; padding: 1em; margin-bottom: 1em; border-radius: 5px; }
        h2 { border-bottom: 1px solid #ccc; padding-bottom: 0.5em; }
        pre { background-color: #f4f4f4; padding: 1em; border-radius: 3px; white-space: pre-wrap; word-wrap: break-word; }
    </style>
</head>
<body>
    <div class="container">
        <h1>RFP RAG PoC</h1>

        <div class="panel">
            <h2>1. Ingest Documents</h2>
            <form id="upload-form" enctype="multipart/form-data">
                <input type="file" name="files" multiple required>
                <button type="submit">Upload and Ingest</button>
            </form>
            <h3>Ingestion Result:</h3>
            <pre id="ingest-result"></pre>
        </div>

        <div class="panel">
            <h2>2. Query</h2>
            <form id="query-form">
                <input type="text" name="question" style="width: 80%;" required>
                <button type="submit">Ask</button>
            </form>
            <h3>Query Result:</h3>
            <pre id="query-result"></pre>
        </div>

        <div class="panel">
            <h2>3. Admin</h2>
            <button id="purge-button">Purge Data</button>
            <h3>Purge Result:</h3>
            <pre id="purge-result"></pre>
        </div>
    </div>

    <script>
        document.getElementById('upload-form').addEventListener('submit', async (e) => {
            e.preventDefault();
            const formData = new FormData(e.target);
            const response = await fetch('/ingest/upload', { method: 'POST', body: formData });
            const result = await response.json();
            document.getElementById('ingest-result').textContent = JSON.stringify(result, null, 2);
        });

        document.getElementById('query-form').addEventListener('submit', async (e) => {
            e.preventDefault();
            const question = e.target.elements.question.value;
            const response = await fetch('/query', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ question })
            });
            const result = await response.json();
            document.getElementById('query-result').textContent = JSON.stringify(result, null, 2);
        });

        document.getElementById('purge-button').addEventListener('click', async () => {
            if (confirm('Are you sure you want to delete all data?')) {
                const response = await fetch('/purge', { 
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ confirm: true })
                });
                const result = await response.json();
                document.getElementById('purge-result').textContent = JSON.stringify(result, null, 2);
            }
        });
    </script>
</body>
</html>
"""

@app.get("/", response_class=HTMLResponse)
async def get_ui():
    return html_ui

@app.post("/ingest/upload")
async def ingest_upload(files: List[UploadFile] = File(...)):
    if len(files) > 2:
        raise HTTPException(status_code=400, detail="Cannot upload more than 2 files.")
    
    module_store = read_module_store()
    newly_created_modules = {}

    for file in files:
        try:
            contents = await file.read()
            text = ""
            if file.filename.endswith('.pdf'):
                with pdfplumber.open(io.BytesIO(contents)) as pdf:
                    text = "\n".join(page.extract_text() for page in pdf.pages)
            else:
                text = contents.decode('utf-8')
            
            modules = distill_document_agent(text, file.filename)
            module_store.extend(modules)
            newly_created_modules[file.filename] = [m['module_id'] for m in modules]

        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Failed to process {file.filename}: {e}")
    
    write_module_store(module_store)
    return JSONResponse(content={"created_modules": newly_created_modules})

@app.get("/modules")
async def get_modules():
    return read_module_store()

@app.post("/query")
async def query(request: QueryRequest):
    module_store = read_module_store()
    retrieval_result = retrieve_modules_agent(request.question, module_store)
    
    if retrieval_result['status'] == 'none':
        return {"retrieval": retrieval_result, "assembly": None}
    
    selected_modules = [m for m in module_store if m['module_id'] in retrieval_result['selected_module_ids']]
    assembly_result = assemble_answer_agent(request.question, selected_modules)
    
    return {"retrieval": retrieval_result, "assembly": assembly_result}

@app.post("/purge")
async def purge(payload: Dict[str, bool]):
    if not payload.get('confirm'):
        raise HTTPException(status_code=400, detail="Confirmation not received.")
    
    deleted_files = []
    if os.path.exists(MODULE_STORE_FILE):
        os.remove(MODULE_STORE_FILE)
        deleted_files.append(MODULE_STORE_FILE)
    if os.path.exists(LOGS_FILE):
        os.remove(LOGS_FILE)
        deleted_files.append(LOGS_FILE)
        
    return {"status": "purged", "deleted_files": deleted_files}


## 4. Launch and Expose the Service

In [ ]:
import uvicorn
import threading
from google.colab import output
import nest_asyncio

def run_app():
  # nest_asyncio is needed because Colab runs an event loop and uvicorn needs to run its own.
  nest_asyncio.apply()
  uvicorn.run(app, host='0.0.0.0', port=8000)

# Run the app in a background thread to keep the notebook interactive
thread = threading.Thread(target=run_app)
thread.start()

# Use colab's output to generate a public URL for the running app
print(f"Access the UI via the following public URL:")
print(output.eval_js('google.colab.kernel.proxyPort(8000)'))